In [16]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torch.autograd import Variable
from csv import reader
import os
import matplotlib.image as mpimg
from PIL import Image


# dataset = Dataset.ImageFolder('Pokemon_DB/images/', transform=transform)


class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # store the inputs and outputs
        self.dataset = self.load_csv(path)[1:]
        self.transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])
        # convert col 1 of names to image filepaths
        files = os.listdir('Pokemon_DB/images/')   
        for i, row in enumerate(self.dataset):
            # find filepath - could be png or jpg
            if row[0] + '.png' in files:
                self.dataset[i][0] = np.asarray(mpimg.imread('Pokemon_DB/images/' + row[0] + '.png') * 255).astype(np.uint8)
            else:
                self.dataset[i][0] = np.asarray(mpimg.imread('Pokemon_DB/images/' + row[0] + '.jpg') * 255).astype(np.uint8)
            self.dataset[i][0] = Image.fromarray(self.dataset[i][0])
            self.dataset[i][0] = self.transform(self.dataset[i][0])

        
        self.rows = len(self.dataset)
        self.cols = len(self.dataset[0])
 
    # Load a CSV file
    def load_csv(self, filename):
        file = open(filename, "r")
        lines = reader(file)
        dataset = list(lines)
        return dataset

    # number of rows in the dataset
    def __len__(self):
        return self.rows
 
    # get a row at an index
    def __getitem__(self, idx):
        return (self.dataset[idx][0], self.dataset[idx][1])



In [17]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [18]:
# create the dataset
# Load dataset
path = './Pokemon_DB/pokemon.csv'
data = CSVDataset(path)

# select rows from the dataset
train, test = random_split(data, [len(data) - 100, 100])


# create a data loader for train and test sets
train_dl = DataLoader(train, batch_size=32, shuffle=True)
test_dl = DataLoader(test, batch_size=1024, shuffle=False)

In [19]:
# define the model
class CNN(nn.Module):
    # define model elements
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=5, padding=2), 
                                nn.BatchNorm2d(16), 
                                nn.ReLU(), 
                                nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
    
    def Forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [20]:
cnn = CNN()

In [21]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [22]:
print(train_dl)
# train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dl):
        print(torch.from_numpy(np.asarray(images)))
        images = Variable(torch.from_numpy(np.asarray(images)))
        labels = Variable(labels)
        
        #Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = loss(outputs,labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' %(epoch+1, num_epochs, i+1, len(train_dl)//batch_size, loss.data[0]))


RuntimeError: stack expects each tensor to be equal size, but got [4, 224, 224] at entry 0 and [3, 224, 224] at entry 3